# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**
Equipo 9:

Yohanna Ceballos Salomón | A01795115

Mauricio Castilo Galindo | A01795453

Juan Manuel Carballo M. | A01166758

Andrea Cantú Martínez | A01235000

Armando Cerda De La Rosa | A01570376


In [ ]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import os

import re
import string
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wnl = WordNetLemmatizer()

from collections import Counter

!pip install fasttext
import fasttext
import fasttext.util

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.


Descarga los 3 archivos de Canvas y genera un solo DataFrame de Pandas con ellos. En particular, el
archivo de datos de IMDb ya no requiere transformarse para obtener sus 1000 registros. Verifica
que tienes los 3000 registros con sus respectivas etiquetas en dicho DataFrame. Los archivos los
encuentras en Canvas y se llaman: amazon5.txt, imdb5.txt, yelp5.txt.


In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Drive
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/Procesamiento de Lenguaje Natural"
os.chdir(DIR)

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', sep='\s{3,}', names=['review','label'], header=None, encoding='utf-8', engine='python')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')

print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

df = pd.concat([dfa, dfi, dfy], ignore_index=True)

print("\n")
print(df['label'].value_counts())

# *********** Aquí termina la sección de agregar código *************

Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)


label
0    1500
1    1500
Name: count, dtype: int64


In [ ]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
# Los dos siguientes comentarios se eliminan con la limpieza posterior a realizar.
# Esto debido a que sólo son dos comentarios y no afecta el modelo.

print(df.iloc[1125,:])
print("\n")
print(df.iloc[1788,:])

review    10/10
label         1
Name: 1125, dtype: object


review    10/10
label         1
Name: 1788, dtype: object


# Pregunta #2

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.

Realiza un proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado a los comentarios, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización,pero no de stemming. Aplica y justifica cualquier otro proceso de limpieza que consideres
adecuado.

Llamar Xclean a los comentarios procesados y Y a las etiquetas.

NOTA: Como aplicaremos modelos embebidos pre-entrenados, queremos palabras lo más cercanas a las existentes en un idioma, inglés en este caso, por ello la técnica de lematización y no de stemming.


In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Separar la información
#     La "X" son los comentarios.
#     La "Y" son la evaluación.
# Ambos, X y Y son "Series"

X = df.review.copy() # Serie de strings
Y = df.label.copy() # Serie de enteros 0s y 1s

assert X.shape == (3000,)           # verificando que tenemos la dimensiones esperadas.
assert Y.shape == (3000,)

# Stopwords
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

mystopwords = set(stopwords.words("english"))
mystopwords = [w for w in mystopwords if w not in negwords]
print("\n Longitud de mystopwords: ", len(mystopwords))

def clean_tok(doc):
  assert type(doc) == str

  # Cambiar todo a minúsculas
    # Justificación:
      # Se cambió todo a minúsculas para tomar en cuenta el uso de las mismas palabras y que no afecte el tener mayúsculas o minúsculas.
      # De esta manera, se si hay una palabra que tiene minúsculas y la misma palabra pero con una letra en mayúsculas, se toma como si fuera la misma palabra.
  tokens = doc.lower()

  # Agregar un espacio después de los signos de puntuación
    # Justificación:
      # Hay oraciones en donde no hay un epacio entre el punto y la siguiente palabra, por lo que toma un token como si fuera una misma,
      # cuando en realidad deberían de ser 2 tokens en vez de 1.
  tokens = re.sub(r'\b[\.]\b', ' ', tokens)

  # Solo considerar caracteres alfabéticos
    # Justificación:
      # El considerar solo los caracteres alfabéticos nos ayuda a no tomar en cuenta los signos de puntuación o números, ya que en este caso no los necesitamos
      # para nuestro análisis. Además nos ayuda a reducir nuestro vocabulario para usar los caracteres que nos ayudan en nuestro análisis, en este caso los alfabéticos.
  tokens = re.sub(r'[^a-z\s]', '  ', tokens)
  tokens = re.sub(r'[\s]{2,}', ' ', tokens)
  tokens = tokens.strip()

  # Tokenización, longitud mayor a 1, eliminar stopwords y eliminar duplicados
     # Justificación:
      # Separamos las oraciones por palabras, ya que estas son las que queremos analizar.
      # Borramos los stopwords, ya que son palabras que se repiten varias veces y no aportan significado a nuestra oración, como in, the, her, him, he, is etc.
      # Al estar hablando de reseñas, no borramos las palabras negativas, ya que nos puede afectar como está escrito un comentario y cambiar su significado.
      # Tomamos en cuenta solo las palabras que su longitud sea mayor a 1, ya que este tipo de palabras normalmente no nos dicen mucho, así como I, A, etc.
      # Después de hacer el procesamiento anterior, es posible nos queden palabras duplicadas, por lo que las borramos.
  tokens = [word for word in tokens.split() if word not in mystopwords]
  tokens = list(set([word for word in tokens if len(word) > 1]))

  # Sustituir los caracteres que aparecen más de 2 veces a 2 veces
    #Justificacion:
      # Tomar las palabras que tienen repeticiones de una letra, cuando no debería de tenerla, así como goooooood, es igual que good.
      # De esta manera podemos reducir las letras que se repiten para poder tomar estas palabras como la misma y las podamos usar en nuestro análisis.
  tokens = [re.sub(r"(.)\1{2,}", r"\1\1", word) for word in tokens]

  # Limpieza adicional
    #Justificacion:
      # En caso dado el patrón anterior no haga match, a continuación el patrón específico para las dos palabras más típicas en reseñas.
      # De esta manera podemos reducir las letras que se repiten para poder tomar estas palabras como la misma y las podamos usar en nuestro análisis.
  tokens = [re.sub(r"\b[g]+[o]{2,}[d]+\b", r"good", word) for word in tokens]
  tokens = [re.sub(r"\b[b]+[a]+[d]+\b", r"bad", word) for word in tokens]

  # Usar Lemmatizaion
    #Justificación:
      # Tenemos variaciones de la misma palabra en el df, por lo que podemos tomar la palabra raíz para tomar estas variaciones como la misma palabra.
      # Usamos Lemmatization, ya que usaremos modelos embebidos pre-entrenados más adelante
    # Extraer lema o lematización para verbos
  tokens = [wnl.lemmatize(word, pos='v') for word in tokens]
    # Extraer lema o lematización para sustantivos
  tokens = [wnl.lemmatize(word, pos='n') for word in tokens]
    # Extraer lema o lematización para adjetivos
  tokens = [wnl.lemmatize(word, pos='a') for word in tokens]
    # Extraer lema o lematización para adverbios
  tokens = [wnl.lemmatize(word, pos='r') for word in tokens]
    # Verificar si después del proceso anterior resulta algún stopword
  tokens = [word for word in tokens if word not in mystopwords]
    # Longitud mayor a 1 y eliminamos palabras duplicadas
  tokens = list(set([word for word in tokens if len(word) > 1]))

  return tokens

# Aplicamos el proceso de limpieza y tokenización:
Xclean = [clean_tok(x) for x in X]


# *********** Aquí termina la sección de agregar código *************


 Longitud de mystopwords:  139


In [ ]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)

['no', 'way', 'plug', 'go', 'converter', 'unless']
['excellent', 'good', 'case', 'value']
['jawbone', 'great']
['conversation', 'charger', 'tie', 'major', 'minute', 'problem', 'last']
['great', 'mic']


# Pregunta #3
Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder
comparar dichos resultados con los de esta actividad, a saber, 70%, 15% y 15%, para
entrenamiento, validación y prueba, respectivamente. Verifica que obtienes 2100 registros de
entrenamiento y 450 para cada uno de validación y prueba.

In [ ]:
# ************* Inicia la sección de agregar código:*****************************


# Partición de los datos
x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

# Verificar las formas de los conjuntos de datos
assert len(x_train) == 2100
assert len(x_val) == 450
assert len(x_test) == 450


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# Pregunta #4
 Construye tu vocabulario a continuación:


In [ ]:
# a.    Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

raw_vocab = Counter()

for k in range(len(x_train)):
  raw_vocab.update(x_train[k])

# Mostrar las 10 palabras más frecuentes y el tamaño del vocabulario inicial
print(f"Longitud del diccionario inicial: {len(raw_vocab)}")
print(f"10 palabras más frecuentes: {raw_vocab.most_common(10)}")

# Definir la frecuencia mínima y longitud mínima
  # Justificación:
    # Se realizaron pruebas con diferentes frecuencias de palabras, pero la mejor fue con 3, pues a medida que aumentamos, quedan muchos comentarios vacíos.
    # Y a medida que se disminuye, agrega ruido al modelo y no se obtiene buenos resultados.
    # Respecto a la longitud mínima de la palabra, optamos por 2 debido a que la palabra "no" es importante en este contexto y tiene apenas 2 de longitud.
min_freq = 3
min_length = 2

# Filtrar el vocabulario
filtered_vocab = {word for word, freq in raw_vocab.items() if freq >= min_freq and len(word) >= min_length}

# *********** Aquí termina la sección de agregar código *************

Longitud del diccionario inicial: 3237
10 palabras más frecuentes: [('not', 205), ('good', 193), ('great', 134), ('movie', 130), ('phone', 123), ('film', 121), ('work', 109), ('bad', 101), ('time', 99), ('like', 94)]


In [ ]:
# b.    Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


len(filtered_vocab)


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:


911

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

 *  **Generalización:** Usar únicamente el conjunto de entrenamiento asegura que el modelo solo tenga acceso a información que estaría disponible durante el entrenamiento y evita cualquier tipo de información de fuga desde los conjuntos de validación y prueba.
 *  **Evaluación justa:** Mantener los conjuntos de validación y prueba independientes asegura que estos datos no influencien la creación del vocabulario y permite una evaluación justa y realista del rendimiento del modelo en datos no vistos.
 *  **Reproducibilidad:** Utilizando solo el conjunto de entrenamiento para construir el vocabulario, se garantiza que el proceso sea repetible y consistente, incluso cuando se cambien los datos de validación y prueba.

### ++++++++ Termina la sección de agregar texto: +++++++++++

In [ ]:
# d.    Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Función para filtrar los datos basados en el vocabulario
def filter_by_vocab(data, vocab):
    return [[word for word in doc if word in vocab] for doc in data]

# Filtrar los conjuntos de datos
train_x = filter_by_vocab(x_train, filtered_vocab)
val_x = filter_by_vocab(x_val, filtered_vocab)
test_x = filter_by_vocab(x_test, filtered_vocab)

# Filtro para borrar los comentarios vacios
def rmv_empty_rev(x_lst, y_series):
  df_tmp = pd.DataFrame({'review': [" ".join(w) for w in x_lst], 'label': y_series})
  df_tmp = df_tmp[df_tmp['review'] != ""]
  x_lst = df_tmp.review
  x_lst = [rev.split() for rev in x_lst]
  y_series = df_tmp.label

  return x_lst, y_series

train_x, y_train = rmv_empty_rev(train_x, y_train)
val_x, y_val = rmv_empty_rev(val_x, y_val)
test_x, y_test = rmv_empty_rev(test_x, y_test)

print('X,y Train:', len(train_x), len(y_train))
print('X,y Val:', len(val_x), len(y_val))
print('X,y Test', len(test_x), len(y_test))

# Mostrar algunos ejemplos para verificar
print(f"\nFirst 5 filtered training examples: {train_x[:5]}")
print(f"First 5 filtered validation examples: {val_x[:5]}")
print(f"First 5 filtered test examples: {test_x[:5]}")


# *********** Aquí termina la sección de agregar código *************

X,y Train: 2084 2084
X,y Val: 440 440
X,y Test 438 438

First 5 filtered training examples: [['like', 'star', 'ed', 'good', 'waste', 'much', 'people', 'don'], ['suck', 'special'], ['server', 'pay', 'job', 'tip', 'not', 'bill', 'terrible', 'felt'], ['steak', 'cook', 'understand', 'call', 'don'], ['wrong', 'however', 'sometimes', 'button', 'keypad', 'tinny']]
First 5 filtered validation examples: [['battery', 'completely', 'useless'], ['friendly', 'service', 'super'], ['try', 'make', 'call'], ['atmosphere', 'good'], ['turn', 'buy', 'else', 'someone', 'back', 'definitely', 'unless']]
First 5 filtered test examples: [['absolutely'], ['plot', 'no', 'first', 'go', 'place', 'keep'], ['time', 'also', 'quick', 'ship'], ['overall'], ['without', 'effect', 'drop']]


Hasta este punto básicamente has realizado transformaciones muy análogas a las de la semana
pasada y que son válidas para muchos de los procesos dentro del análisis de textos.

Procedamosahora con los vectores embebidos preentrenados de cada palabra, en lugar de los vectores
generados con las matrices Tf-idf que utilizaste la semana pasada.


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['like', 'star', 'ed', 'good', 'waste', 'much', 'people', 'don']
['suck', 'special']
['server', 'pay', 'job', 'tip', 'not', 'bill', 'terrible', 'felt']
['steak', 'cook', 'understand', 'call', 'don']
['wrong', 'however', 'sometimes', 'button', 'keypad', 'tinny']


# Pregunta #5
En particular, utilizaremos los vectores embebidos FastText preentrenados por Facebook.

a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

 Puedes consultar sus páginas correspondientes:

https://fasttext.cc/

https://code.google.com/archive/p/word2vec/

https://nlp.stanford.edu/projects/glove/


### ++++++++ Inicia la sección de agregar texto: +++++++++++

<table>
  <tr>
    <th>Embedding</th>
    <th>Pros</th>
    <th>Cons</th>
  </tr>
  <tr>
    <td>Word2Vec</td>
    <td>
      <ul>
        <li>Captura relaciones semanticas efectivamente</li>
        <li>Eficiente para datasets muy grandes</li>
        <li>Provee representaciones de palabras significativas</li>
      </ul>
    </td>
    <td>
      <ul>
        <li>Tiende a equivocarse con palabras no ordinarias o raras</li>
        <li>Ignora el orden de las palabras</li>
      </ul>
    </td>
  </tr>
  <tr>
    <td>GloVe</td>
    <td>
      <ul>
        <li>Efectivo para capturar estadísticas globales del corpus</li>
        <li>Bueno para representar relaciones semanticas y sintacticas del corpus</li>
        <li>Efectivo para capturar analogias de las palabras</li>
      </ul>
    </td>
    <td>
      <ul>
        <li>Require más memoria para almacenar la matrix co-ocurrencia</li>
        <li>Menos efectivo con corpus muy pequeños</li>
      </ul>
    </td>
  </tr>
  <tr>
    <td>FastText</td>
    <td>
      <ul>
        <li>Bueno para representar palabras no ordinarias o raras</li>
        <li>Capaz de manejar palabras no existentes en el vocabulario</li>
        <li>Representacion de palabras enriquecidas debido su descomposición en n-grams</li>
      </ul>
    </td>
    <td>
      <ul>
        <li>El tamaño del modelo es más grande debido a la información de los n-grams</li>
        <li>Periodos de entramiento más largos comparados a Word2Vec</li>
      </ul>
    </td>
  </tr>
</table>

### ++++++++ Termina la sección de agregar texto: +++++++++++

#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



NOTA1: Puedes consultar la documentación: https://fasttext.cc/docs/en/crawl-vectors.html

NOTA2: Debido a la cantidad de recursos computacionales que demanda cargar los vectores
FastText (son 2 millones de vectores), es recomendable que una vez que generes el nuevo
vocabulario de vectores embebidos, guardes dicho diccionario en un archivo (pickle, npz o el que
consideres más adecuado). Una vez realizado lo anterior, puedes borrar la variable de FastText
para liberar memoria RAM. Apóyense entre los miembros del equipo si alguno de ustedes tiene
problemas de recursos computacionales para cargar los vectores embebidos.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

#Si no se tiene el archivo 'cc.en.300.bin' en la ruta inicial, ejecutar el siguiente comando
#y ubicarlo en la ruta "/content/drive/MyDrive/Colab Notebooks/MNA/Procesamiento de Lenguaje Natural"

#fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

midiccEmb_vector = {}
for word in filtered_vocab:
  tmp = ft.get_word_vector(word)
  midiccEmb_vector.update({word: tmp})

# *********** Aquí termina la sección de agregar código *************

# Pregunta #7

Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.
Una manera de utilizar los vectores embebidos con modelos de aprendizaje automático (machine learning) en documentos de texto, es asignar a cada comentario el vector embebido de dimensión predeterminada que resulta de promediar todos los vectores embebidos de cada una de sus palabras (tokens).

Así, en este ejercicio deberás generar los conjuntos de entrenamiento, validación y prueba de de esta manera. Los llamaremos trainEmb, valEmb y testEmb, respectivamente. Es decir, ahora cada comentario es un solo comentario de dimensión 300.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


trainEmb = np.array([np.mean(np.array([midiccEmb_vector[word] for word in sent]), axis=0) for sent in train_x])
valEmb = np.array([np.mean(np.array([midiccEmb_vector[word] for word in sent]), axis=0) for sent in val_x])
testEmb = np.array([np.mean(np.array([midiccEmb_vector[word] for word in sent]), axis=0) for sent in test_x])


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:
print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2084, 300)
Val-Emb: (440, 300)
Test-Emb: (438, 300)


# Pregunta #8

Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.


Utilizando los nuevos conjuntos embebidos de entrenamiento y validación, obtener los modelos de
regresión logística y bosque aleatorio (random forest). Para cada modelo muestra el valor de la
exactitud (accuracy) y el reporte de sklearn dado por la función classification_report(). Verifica que
no estén sobreentrenados y compara tus resultados con los que obtuviste en la actividad de la
semana pasada

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


# Inicializamos el modelo de LR
modeloLR = LogisticRegression(C=11, max_iter=1000)
# Entrenamos con los datos de entrenamiento DTM
modeloLR.fit(trainEmb, y_train)
# Realizamos las predicciones con los datos de validación transformados
yhat = modeloLR.predict(valEmb)

print('LR: Train-accuracy: %.2f%%' % (100*modeloLR.score(trainEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloLR.score(valEmb, y_val)))
print("\n")
print(classification_report(y_val, yhat, zero_division=0))
# *********** Aquí termina la sección de agregar código *************

LR: Train-accuracy: 86.18%
LR: Val-accuracy: 84%


              precision    recall  f1-score   support

           0       0.82      0.84      0.83       211
           1       0.85      0.83      0.84       229

    accuracy                           0.84       440
   macro avg       0.84      0.84      0.84       440
weighted avg       0.84      0.84      0.84       440



In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


# Inicializamos el modelo de RF
modeloRF = RandomForestClassifier(n_estimators = 69,
                                  max_depth=2,
                                  bootstrap=True,
                                  random_state=11,
                                  min_samples_split = 2,
                                  min_samples_leaf = 111,
                                  max_features ='sqrt'
                                  )
# Entrenamos con los datos de entrenamiento DTM
modeloRF.fit(trainEmb, y_train)
# Realizamos las predicciones con los datos de validación transformados
yhat = modeloRF.predict(valEmb)

print('\nRF: Train-accuracy: %.2f%%' % (100*modeloRF.score(trainEmb, y_train)))
print('RF: Val-accuracy: %.2f%%' % (100*modeloRF.score(valEmb, y_val)))
print("\n")
print(classification_report(y_val, yhat, zero_division=0))


# *********** Aquí termina la sección de agregar código *************


RF: Train-accuracy: 79.89%
RF: Val-accuracy: 76.82%


              precision    recall  f1-score   support

           0       0.73      0.82      0.77       211
           1       0.81      0.72      0.76       229

    accuracy                           0.77       440
   macro avg       0.77      0.77      0.77       440
weighted avg       0.77      0.77      0.77       440



> Como regla general, se busca que la diferencia entre la precisión en entrenamiento y validación sea lo más pequeña posible. En este caso, dado el tamaño del conjunto de datos, la complejidad del modelo y la naturaleza del problema, definimos esa diferencia máxima de 3% para que no se considere sobreentrenamiento. En particular, en los dos modelos usados anteriormente, la diferencia es menos del 3%, por lo que descartamos sobreentrenamiento.

> En comparación con los resultados de la semana pasada, esta semana se obtienen valores ligeramente superiores. Sin embargo, se esperaban resultados más altos en comparación con las matrices de conteo y tf-idf dado a que FastText es un modelo pre-entrenado.

# Pregunta #9
Con el mejor modelo y el nuevo conjunto de prueba, obtener la mejor matriz de confusión y el
classification_report() de sklearn.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


mejor_modelo = modeloLR

print('Test-accuracy con el mejor modelo %.2f%%' % (100*mejor_modelo.score(testEmb, y_test)))

pred = mejor_modelo.predict(testEmb)
print('\nMatriz de confusión con el mejor modelo:')
print(confusion_matrix(y_test, pred, labels=[0,1]))

print('\nMatriz de confusión con el mejor modelo en proporciones:')
print(confusion_matrix(y_test, pred, labels=[0,1]) / pred.shape[0])

print("\n")
print(classification_report(y_test, pred, zero_division=0))


# *********** Aquí termina la sección de agregar código *************

Test-accuracy con el mejor modelo 82.88%

Matriz de confusión con el mejor modelo:
[[175  33]
 [ 42 188]]

Matriz de confusión con el mejor modelo en proporciones:
[[0.39954338 0.07534247]
 [0.09589041 0.42922374]]


              precision    recall  f1-score   support

           0       0.81      0.84      0.82       208
           1       0.85      0.82      0.83       230

    accuracy                           0.83       438
   macro avg       0.83      0.83      0.83       438
weighted avg       0.83      0.83      0.83       438



# Pregunta #10
Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

En este notebook, implementamos y comparamos dos modelos de aprendizaje automático para la clasificación de textos: Regresión Logística (LR) y Bosques Aleatorios (RF). La metodología incluyó varios pasos clave, como la preprocesamiento de datos, la generación de embeddings usando FastText, el entrenamiento de modelos y la evaluación del rendimiento.

Primero, preprocesamos los datos de texto limpiándolos y tokenizándolos, y luego generamos embeddings de palabras con el modelo preentrenado FastText. Esto nos permitió utilizar vectores de características ricos y de alta dimensión para cada palabra, capturando tanto información sintáctica como semántica.

Luego, entrenamos los modelos de Regresión Logística y Bosques Aleatorios usando estos embeddings. Para la Regresión Logística, observamos una precisión de entrenamiento del 86.18% y una precisión de validación del 84%, con precisión, recall y F1-score relativamente equilibrados en ambas clases. El modelo de Bosques Aleatorios logró precisiones de entrenamiento y validación ligeramente más bajas del 79.89% y 76.82%, respectivamente.

Comparando los dos modelos, la Regresión Logística mostró métricas de rendimiento ligeramente mejores. Su mayor precisión y métricas de precisión, recall y F1-score más equilibradas sugieren que es mejor para generalizar a datos no vistos en comparación con el modelo de Bosques Aleatorios. Además, el modelo de Regresión Logística mostró un sobreajuste mínimo, indicado por la pequeña diferencia entre las precisiones de entrenamiento y validación. Finalmente, el modelo de Regresión Logística evaluado en el conjunto de entrenamiento, logró un satisfactorio 82.88% de precisión, lo cual indica una ligera mejoría respecto a la actividad de la semana pasada.

En general, se presume que entre más bajo es min_freq se introduce más ruido al modelo, lo que hace que la precisión no sea la mejor, es decir, el modelo no aprende a generalizar cuando se evalua con el set de pruebas, pues no se obtienen buenos resultados. Sin embargo, cuando se toman valores de más de 5 en min_freq, se obtiene una gran cantidad de comentarios vacíos en el conjunto de entrenamiento, lo cual deteriora el desempeño del modelo. La disparidad entre los valores de Falsos Negativos y Falsos Positivos se debe principalmente a la falta de contextualización en los métodos utilizados para generar las representaciones vectoriales de las palabras en los comentarios. Adicional, dependiendo del orden y los pasos de la limpieza llevado a cabo en los primeros puntos, el desempeño del modelo mejora o empeora, por lo que es fundamental concentrarse en el pre-procesamiento.


En conclusión, la metodología de usar embeddings de FastText proporcionó una base sólida para ambos modelos, pero el modelo de Regresión Logística resultó ser la mejor opción para esta tarea de clasificación de textos. Su capacidad para generalizar bien, combinada con métricas de rendimiento consistentes, lo convierte en un modelo más fiable para aplicaciones futuras. Este análisis comparativo destaca la importancia de evaluar múltiples modelos y metodologías para identificar el enfoque más efectivo para un problema dado, es decir hacer mucho testing.

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**

